## Spark Wrangle Exercises:
### Corey Solitaire
`11.30.2020`

In [1]:
# Create Spark Session
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

'''at a company this would be more complex, and include
commands to set up specific enfiroments'''

spark = SparkSession.builder.getOrCreate()

### To schema or not to schema

 - Schema: a way of specifying the datatypes / data shape to spark   
 
 - Why:
   - Preformance
   - Data Integrity

In [2]:
# Call in DF with specified structures
# header = True (calls in headers from CSF)
# inferSchema = True (infers datatype of columns in csv)
source = spark.read.csv('source.csv', header = True, inferSchema = True)
dept = spark.read.csv('dept.csv', header = True, inferSchema = True)
case = spark.read.csv('case.csv', header = True, inferSchema = True)

***
## Exercises:
- This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.   

`You might have noticed that the latest date in the dataset is fairly far off from the present day. To account for this, replace any occurances of the current time with the maximum date from the dataset.`
***

### 1. Read the case, department, and source data into their own spark dataframes.

In [3]:
print(source.show(5)),print(dept.show(5)),print(case.show(5))

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
|   119403|  Betty De Hoyos|
|   119555|  Socorro Quiara|
+---------+----------------+
only showing top 5 rows

None
+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
+------------

(None, None, None)

### 2. Let's see how writing to the local disk works in spark:

- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

- Inspect your folder structure. What do you notice?

In [4]:
source.write.csv("source_csv", mode="overwrite")
dept.write.csv("dept_csv", mode="overwrite")
case.write.csv("case_csv", mode="overwrite")
source.write.json("source_json", mode="overwrite")
dept.write.json("dept_json", mode="overwrite")
case.write.json("case_json", mode="overwrite")

***
`Inside the folder the csv and json objects are stored using unique identifiers`
***

### 3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [5]:
source.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



In [6]:
dept.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



In [7]:
case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



### Changes to case data frame

In [8]:
# rename to match other columns
case = case.withColumnRenamed("SLA_due_date", "case_due_date")

# change case_closed and case_late to boolean values
case = case.withColumn("case_closed", expr('case_closed == "YES"')).withColumn(
    "case_late", expr('case_late == "YES"')
)

# change city coulncil district to string not int (categorical)
case = case.withColumn("council_district", col("council_district").cast("string"))
# add 00 before number
case = case.withColumn("council_district", col("council_district").cast("int"))

# '%03d' means at least 3 digits, pad with 0s
#
# In order to use the format_string function the way we are, we'll need to
# convert council_district back to an integer temporarily, but the final output
# will be a string.
case = case.withColumn(
    "council_district",
    format_string("%03d", col("council_district").cast("int")),
)

# Now we will handle the 3 columns that have dates in them. We'll use spark's to_timestamp function for this.
fmt = "M/d/yy H:mm"
case = (
    case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_due_date", fmt))
)

# standardize address column, trim, removes whitespace, lowercase
case = case.withColumn("request_address", trim(lower(case.request_address)))

# convert number of days late to weeks
case = case.withColumn(
    "num_weeks_late", expr("num_days_late / 7 AS num_weeks_late")
)

# new features (zipcode)
case = case.withColumn("zipcode", regexp_extract("request_address", r"\d+$", 0))

''' 
Next we will create several new, related columns:

- case_age: How old the case is; the difference in days between when the case was opened and the current day
- days_to_closed: The number of days between when the case was opened and when it was closed
- case_lifetime: Number of days between when the case was opened and when it was closed, if the case is still open, the number of days since the case was opened

'''

case = (
    case.withColumn(
        "case_age", datediff(current_timestamp(), "case_opened_date")
    )
    .withColumn(
        "days_to_closed", datediff("case_closed_date", "case_opened_date")
    )
    .withColumn(
        "case_lifetime",
        when(expr("! case_closed"), col("case_age")).otherwise(
            col("days_to_closed")
        ),
    )
)

In [9]:
#Joint modified data frames

df = (
    case
    # left join on dept_division
    .join(dept, "dept_division", "left")
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(dept.dept_division)
    .drop(dept.dept_name)
    .drop(case.dept_division)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
)

In [10]:
# Check to see if it works
df.show(2, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 1065                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
-RECORD 1-------------------------

### Train / Test Split

In [11]:
# train, test = df.randomSplit([0.8, 0.2],seed=123)
# train, validate, test = df.randomSplit([0.6, 0.2, 0.2],seed=123)

In [12]:
# verify split train
# print((train.count(), len(train.columns)))

In [13]:
# verify split validate
# print((validate.count(), len(validate.columns)))

In [14]:
# verify split test
# print((test.count(), len(test.columns)))

*** 
### 1.  How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?
 

In [15]:
df.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = false)
 |-- num_weeks_late: double (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- case_age: integer (nullable = true)
 |-- days_to_closed: integer (nullable = true)
 |-- case_lifetime: integer (nullable = true)
 |-- department: string (nullable = true)
 |-- dept_subject_to_SLA: boolean (nullable = true)



In [16]:
# How old is the latest currently open issue
(
df.select('SLA_days','case_opened_date','case_due_date','case_lifetime','case_age','num_days_late')
  .where(df.case_closed == False)
  .sort(df.case_opened_date.desc())
  .show(2)
)

+--------+-------------------+-------------------+-------------+--------+-------------+
|SLA_days|   case_opened_date|      case_due_date|case_lifetime|case_age|num_days_late|
+--------+-------------------+-------------------+-------------+--------+-------------+
|     9.0|2018-08-08 10:38:00|2018-08-17 10:38:00|          846|     846| -8.443217593|
|    14.0|2018-08-08 10:38:00|2018-08-22 10:38:00|          846|     846| -13.44305556|
+--------+-------------------+-------------------+-------------+--------+-------------+
only showing top 2 rows



In [17]:
# How long has the oldest (in terms of days since opened) currently opened issue been open?¶
(
df.select('SLA_days','case_opened_date','case_due_date','case_lifetime','case_age','num_days_late')
  .where(df.case_closed == False)
  .sort(df.case_opened_date.asc())
  .show(2)
)

+-----------+-------------------+-------------------+-------------+--------+-------------+
|   SLA_days|   case_opened_date|      case_due_date|case_lifetime|case_age|num_days_late|
+-----------+-------------------+-------------------+-------------+--------+-------------+
|15.77859954|2017-01-01 13:48:00|2017-01-17 08:30:00|         1430|    1430|  348.6458333|
|15.77247685|2017-01-01 13:57:00|2017-01-17 08:30:00|         1430|    1430|  348.6458333|
+-----------+-------------------+-------------------+-------------+--------+-------------+
only showing top 2 rows



In [18]:
## Alternate via walkthrough
# How old is the latest (in terms of days past SLA) currently open issue?
# How long has the oldest (in terms of days since opened) currently opened issue been open?
# spark.sql('''
# SELECT DATEDIFF(current_timestamp, case_due_date) AS days_past_due
# FROM df
# WHERE NOT case_closed
# ORDER BY days_past_due DESC
# LIMIT 15
# ''').show()

   
### 2. How many Stray Animal cases are there?

In [19]:
(
df.where(df.department == 'Animal Care Services')
  .agg(count('case_id'))
  .show()
)

+--------------+
|count(case_id)|
+--------------+
|        116915|
+--------------+



In [20]:
# Alternate
(
    df.filter(col("service_request_type") == lit("Stray Animal"))
    .groupBy("case_closed", "service_request_type")
    .count()
    .sort(col("count").desc())
    .show()
)

+-----------+--------------------+-----+
|case_closed|service_request_type|count|
+-----------+--------------------+-----+
|       true|        Stray Animal|26745|
|      false|        Stray Animal|   15|
+-----------+--------------------+-----+



### 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [21]:
(
    df.filter(col("service_request_type") != lit("Officer Standby"))
    .count()
)

838691

### 4. Convert the council_district column to a string column.   

In [22]:
# Already preformed this operation in pre-processing
df.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = false)
 |-- num_weeks_late: double (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- case_age: integer (nullable = true)
 |-- days_to_closed: integer (nullable = true)
 |-- case_lifetime: integer (nullable = true)
 |-- department: string (nullable = true)
 |-- dept_subject_to_SLA: boolean (nullable = true)



### 5. Extract the year from the case_closed_date column.

In [23]:
df = df.withColumn("year", year("case_closed_date"))
#verify
df.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 1065                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 year                 | 2018      

### 6. Convert num_days_late from days to hours in new columns num_hours_late.

In [24]:
df = df.withColumn("num_hours_late", (col("num_days_late") * 24))
# Verify
df.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 1065                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 year                 | 2018      

### 7. Join the case data with the source and department data.

In [25]:


# join source table, dept table already joined
# train = train.join(source, "source_id", "left")

'''
Must add 10141 to the front of source_id to match the case id before join
'''

'\nMust add 10141 to the front of source_id to match the case id before join\n'

In [26]:
# visualize source data frame
source.show(1, vertical = True )

-RECORD 0---------------------------
 source_id       | 100137           
 source_username | Merlene Blodgett 
only showing top 1 row



In [27]:
dept.show(1, vertical = True )

-RECORD 0----------------------------------
 dept_division          | 311 Call Center  
 dept_name              | Customer Service 
 standardized_dept_name | Customer Service 
 dept_subject_to_SLA    | YES              
only showing top 1 row



In [28]:
case.show(1, vertical = True )

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 1065                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
only showing top 1 row



In [29]:
#First step

source = source.select('source_id','source_username').withColumn('source_id',concat(lit("10141"), source.source_id)).withColumnRenamed("source_id","case_id")

In [30]:
# Verify
source.show()

+-----------+--------------------+
|    case_id|     source_username|
+-----------+--------------------+
|10141100137|    Merlene Blodgett|
|10141103582|         Carmen Cura|
|10141106463|     Richard Sanchez|
|10141119403|      Betty De Hoyos|
|10141119555|      Socorro Quiara|
|10141119868| Michelle San Miguel|
|10141120752|      Eva T. Kleiber|
|10141124405|           Lori Lara|
|10141132408|       Leonard Silva|
|10141135723|        Amy Cardenas|
|10141136202|    Michelle Urrutia|
|10141136979|      Leticia Garcia|
|10141137943|    Pamela K. Baccus|
|10141138605|        Marisa Ozuna|
|10141138650|      Kimberly Green|
|10141138650|Kimberly Green-Woods|
|10141138793| Guadalupe Rodriguez|
|10141138810|       Tawona Martin|
|10141139342|     Jessica Mendoza|
|10141139344|        Isis Mendoza|
+-----------+--------------------+
only showing top 20 rows



In [31]:
# source = source.withColumnRenamed("source_id","case_id")

In [32]:
# source.show()

In [33]:
# 2nd Step - Join on df
df = (df.join(source, "case_id", 'left'))

#.join(dept, "dept_division", "left")

In [34]:
df.show(5, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 1065                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 year                 | 2018      

### 8. Are there any cases that do not have a request source?

In [35]:
(
    df.select(df.source_id.isNull().cast('int').alias('is_null'))
    .agg(sum('is_null'))
    .show()
)

+------------+
|sum(is_null)|
+------------+
|           0|
+------------+



In [36]:
df.filter(col('source_id').isNull()).show(vertical=True)

(0 rows)



### 9. What are the top 10 service request types in terms of number of requests?

In [37]:

# What are the top 10 service request types in terms of number of requests?
(
    df.groupby('service_request_type')
    .count()
    .sort(col('count').desc())
    .show(10, truncate=False)
)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |86855|
|Overgrown Yard/Trash            |65895|
|Bandit Signs                    |32910|
|Damaged Cart                    |30338|
|Front Or Side Yard Parking      |28794|
|Stray Animal                    |26760|
|Aggressive Animal(Non-Critical) |24882|
|Cart Exchange Request           |22024|
|Junk Vehicle On Private Property|21473|
|Pot Hole Repair                 |20616|
+--------------------------------+-----+
only showing top 10 rows



### 10. What are the top 10 service request types in terms of average days late?

In [38]:
(
    df.where('case_late') # just the rows where case_late == true
    .groupBy('service_request_type')
    .agg(mean('num_days_late').alias('n_days_late'), count('*').alias('n_cases'))
    .sort(desc('n_days_late'))
    .show(10, truncate=False)
)

+--------------------------------------+------------------+-------+
|service_request_type                  |n_days_late       |n_cases|
+--------------------------------------+------------------+-------+
|Zoning: Recycle Yard                  |210.89201994318182|132    |
|Zoning: Junk Yards                    |200.20517608494276|262    |
|Structure/Housing Maintenance         |190.20707698509807|51     |
|Donation Container Enforcement        |171.09115313942615|122    |
|Storage of Used Mattress              |163.96812829714287|7      |
|Labeling for Used Mattress            |162.43032902285717|7      |
|Record Keeping of Used Mattresses     |153.99724039428568|7      |
|Signage Requied for Sale of Used Mattr|151.63868055333333|12     |
|Traffic Signal Graffiti               |137.64583330000002|2      |
|License Requied Used Mattress Sales   |128.79828704142858|7      |
+--------------------------------------+------------------+-------+
only showing top 10 rows



### 11. Does number of days late depend on department?

In [39]:
(
    df.filter('case_late')
    .groupby('department')
    .agg(mean('num_days_late').alias('days_late'), count('num_days_late').alias('n_cases_late'))
    .sort('days_late')
    .withColumn('days_late', round(col('days_late'), 1))
    .show(truncate=False)
)

+------------------------+---------+------------+
|department              |days_late|n_cases_late|
+------------------------+---------+------------+
|Metro Health            |6.5      |829         |
|Solid Waste             |7.2      |32945       |
|Trans & Cap Improvements|10.6     |5411        |
|Parks & Recreation      |22.3     |3797        |
|Animal Care Services    |23.5     |23276       |
|DSD/Code Enforcement    |49.4     |26235       |
|Customer Service        |87.7     |2010        |
+------------------------+---------+------------+



In [40]:
df.groupby('department').count().show(truncate=False)

+------------------------+------+
|department              |count |
+------------------------+------+
|Solid Waste             |279270|
|Animal Care Services    |116915|
|Trans & Cap Improvements|96193 |
|Parks & Recreation      |19907 |
|Customer Service        |2849  |
|Metro Health            |5163  |
|City Council            |33    |
|DSD/Code Enforcement    |321374|
+------------------------+------+



### 12. How do number of days late depend on department and request type?

In [41]:

# How do number of days late depend on department and request type?
(
    df.filter("case_closed")
    .filter("case_late")
    .groupby("department", "service_request_type")
    .agg(avg("num_days_late").alias("days_late"), count("*").alias("n_cases"))
    .withColumn("days_late", round(col("days_late"), 1))
    .sort(desc("days_late"))
    .show(40, truncate=False)
)

+--------------------+--------------------------------------------+---------+-------+
|department          |service_request_type                        |days_late|n_cases|
+--------------------+--------------------------------------------+---------+-------+
|DSD/Code Enforcement|Zoning: Recycle Yard                        |273.6    |75     |
|DSD/Code Enforcement|Zoning: Junk Yards                          |251.9    |146    |
|DSD/Code Enforcement|Donation Container Enforcement              |201.7    |82     |
|DSD/Code Enforcement|Structure/Housing Maintenance               |182.4    |30     |
|DSD/Code Enforcement|Graffiti: Private Property (Corridors)      |175.1    |3      |
|DSD/Code Enforcement|Storage of Used Mattress                    |164.0    |7      |
|DSD/Code Enforcement|Labeling for Used Mattress                  |162.4    |7      |
|DSD/Code Enforcement|Record Keeping of Used Mattresses           |154.0    |7      |
|DSD/Code Enforcement|Signage Requied for Sale of Used